In [1]:
import tensorflow as tf
from hgru4rec.user_par_mini_batch import input_fn, UserParallelMiniBatchDataset
from tensorflow.contrib.cudnn_rnn import CudnnGRU
from tensorflow.keras.layers import GRU, Dense

In [2]:
tf.enable_eager_execution()

In [3]:
params = dict()
params['num_units_session'] = 25
params['num_units_user'] = 50
params['num_products'] = 2600000
params['embedding_size'] = 25
params['user_rnn_layers'] = 2
params['user_rnn_units'] = 50
params['session_rnn_layers'] = 2
params['session_rnn_units'] = 25
params['num_negative_samples'] = 10

In [4]:
# get datapoint iterator
dataset = input_fn(10, 'gs://ma-muy/baseline_dataset/sessions_by_user/', 3, epochs=2)
datapoints = dataset.make_one_shot_iterator()

Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, use
    tf.py_function, which takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    
Instructions for updating:
Colocations handled automatically by placer.


In [5]:
datapoint = next(datapoints)
features, labels = datapoint
features

/home/muy/.local/share/virtualenvs/code-JIyQgSPM/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/home/muy/.local/share/virtualenvs/code-JIyQgSPM/lib/python3.6/site-packages/google/auth/_default.py:66: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about 

{'UserId': <tf.Tensor: id=254, shape=(10,), dtype=int64, numpy=
 array([1721500,  582800,  898100, 2241000, 2206200, 2802000, 2159700,
        1624500, 1897400,  834800])>,
 'ProductId': <tf.Tensor: id=252, shape=(10,), dtype=int64, numpy=
 array([ 399224, 5949616, 5882871, 9669627, 6012120, 6353176, 7665009,
         432765, 6032131, 6168248])>,
 'EmbeddingId': <tf.Tensor: id=251, shape=(10,), dtype=int64, numpy=array([10, 11, 12,  3, 13,  5, 14, 15,  8, 16])>,
 'UserEmbeddingId': <tf.Tensor: id=253, shape=(10,), dtype=int64, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])>}

In [6]:
# State representation

batch_size = features['UserId'].shape[0]

ended_sessions_mask = tf.get_variable(
        'ended_sessions_mask',
        shape=(batch_size,),
        initializer=tf.zeros_initializer(),
        trainable=False,
        dtype=tf.bool)

ending_sessions_mask = tf.get_variable(
        'continuing_sessions_mask',
        shape=(batch_size,),
        initializer=tf.zeros_initializer(),
        trainable=False,
        dtype=tf.bool)

ended_users_mask = tf.get_variable(
        'ended_users_mask',
        shape=(batch_size,),
        initializer=tf.zeros_initializer(),
        trainable=False,
        dtype=tf.bool)

session_hidden_states = tf.get_variable(
        'session_hidden_states',
        shape=(batch_size, params['session_rnn_units']),
        initializer=tf.zeros_initializer())

user_hidden_states = tf.get_variable(
        'user_hidden_states',
        shape=(batch_size, params['user_rnn_units']),
        initializer=tf.zeros_initializer())

In [7]:
# Embeddings

embeddings = tf.get_variable(
    'embedding',
    shape=(params['num_products'], params['embedding_size']))

softmax_weights = tf.get_variable(
    'softmax_weights',
    shape=(params['num_products'], params['embedding_size']))

softmax_biases = tf.get_variable(
    'softmax_biases',
    shape=(params['num_products'],))

In [8]:
# User & Session RNN

user_rnn = GRU(
    # params['user_rnn_layers'],
    params['user_rnn_units'],
    return_state=True,
    name='user_rnn')

session_rnn = GRU(
    # params['session_rnn_layers'],
    params['session_rnn_units'],
    return_state=True,
    name='session_rnn')

user2session_layer = Dense(params['session_rnn_units'], input_shape=(params['user_rnn_units'],), activation='tanh')

In [9]:
# Reset Session & User Representation for ended users

session_hidden_states = tf.where(
    ended_users_mask,
    tf.zeros(tf.shape(session_hidden_states)),
    session_hidden_states)

user_hidden_states = tf.where(
    ended_users_mask,
    tf.zeros(tf.shape(user_hidden_states)),
    user_hidden_states)

In [10]:
# Predict new Session Representation

new_session_hidden_states_seed, new_user_hidden_states = user_rnn.apply(tf.expand_dims(session_hidden_states, 1), initial_state=user_hidden_states)
new_session_hidden_states = user2session_layer.apply(new_session_hidden_states_seed)

In [11]:
# Select predicted Sessions where necessary

session_hidden_states = tf.where(
    ended_sessions_mask,
    new_session_hidden_states,
    session_hidden_states)

user_hidden_states = tf.where(
    ended_sessions_mask,
    new_user_hidden_states,
    user_hidden_states)

In [12]:
# Compute new masks
ended_sessions_mask = tf.cast(
    tf.where(
        tf.equal(features['ProductId'], -1), 
        tf.ones(tf.shape(ended_sessions_mask)), 
        tf.zeros(tf.shape(ended_sessions_mask))), 
    tf.bool)

ending_sessions_mask = tf.cast(
    tf.where(
        tf.equal(labels['ProductId'], -1), 
        tf.ones(tf.shape(ending_sessions_mask)), 
        tf.zeros(tf.shape(ending_sessions_mask))), 
    tf.bool)

ended_users_mask = tf.cast(
    tf.where(
        tf.equal(features['UserId'], -1), 
        tf.ones(tf.shape(ended_users_mask)), 
        tf.zeros(tf.shape(ended_users_mask))), 
    tf.bool)

In [41]:
# Generate Predictions

relevant_sessions_mask = tf.logical_not(tf.logical_or(ended_sessions_mask, ending_sessions_mask))

relevant_input_embeddings = tf.map_fn(
    lambda x: tf.nn.embedding_lookup(embeddings, x[0])
    if tf.greater(x[1], tf.constant(0, dtype=tf.int64))
    else tf.zeros(params['embedding_size']),
    tf.stack([
        features['EmbeddingId'],
        tf.cast(relevant_sessions_mask, tf.int64)],
        axis=1),
    dtype=tf.float32)

relevant_hidden_states = tf.where(
    relevant_sessions_mask,
    session_hidden_states,
    tf.zeros(tf.shape(session_hidden_states))
)

relevant_labels = tf.boolean_mask(labels['EmbeddingId'], relevant_sessions_mask)

predicted_embeddings, new_session_hidden_states = session_rnn.apply(
    tf.expand_dims(relevant_input_embeddings, 1),
    initial_state=relevant_hidden_states)

predicted_embeddings = tf.boolean_mask(
    predicted_embeddings,
    relevant_sessions_mask)

session_hidden_states = tf.where(
    relevant_sessions_mask,
    new_session_hidden_states,
    session_hidden_states)

softmax_predictions = tf.matmul(predicted_embeddings, softmax_weights, transpose_b=True) + softmax_biases

In [43]:
# Compute Hitrate

in_top_k = tf.nn.in_top_k(softmax_predictions, relevant_labels, 5)
hitrate = tf.divide(
    tf.reduce_sum(tf.cast(in_top_k, tf.int64)),
    tf.shape(labels)[0])

In [47]:
# Compute Loss Function

negative_samples_weights = tf.nn.embedding_lookup(softmax_weights, relevant_labels)
negative_samples_biases = tf.nn.embedding_lookup(softmax_biases, relevant_labels)

logits = tf.matmul(predicted_embeddings, negative_samples_weights, transpose_b=True) + negative_samples_biases
yhat = tf.nn.softmax(logits) # for each of the examples in the batch we select the remainder of the minibatch as negative examples

# TOP 1 Loss function
yhatT = tf.transpose(yhat)
term1 = tf.reduce_mean(tf.nn.sigmoid(-tf.diag_part(yhat)+yhatT)+tf.nn.sigmoid(yhatT**2), axis=0)
term2 = tf.nn.sigmoid(tf.diag_part(yhat)**2) / batch_size.value
loss = tf.reduce_mean(term1 - term2)

In [48]:
predi

<tf.Tensor: id=2867, shape=(), dtype=float32, numpy=0.9640579>

In [55]:
# Optimize
optimizer = tf.train.AdamOptimizer()
train_op = optimizer.minimize(loss, global_step=tf.train.get_or_create_global_step())

RuntimeError: `loss` passed to Optimizer.compute_gradients should be a function when eager execution is enabled.

In [16]:
session_hidden_states

<tf.Tensor: id=639, shape=(10, 25), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
      